-clean up notebook 
-add a hurricane or two 
-run logistics regression with train/test split 

In [1]:
#Importing libraries needed
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## Zillow Data

### Median home value from Kaggle

Just zillow 
-create a dataframe from all of the hurricane data that has target variable (percent increase and yes or no), value of homes 4 months before hurricane, city names, which hurricane, and size ranks

-use statsmodel or scikit learn to do logistic regression on categorical column 4 month before the hurricane (predictnig the value)

## Data Exploration

In [2]:
#opening the dataset to create a dataframe for logistic regression 
value = pd.read_csv(r'data\Sale_Prices_City.csv')
value.head()

,Unnamed: 0,RegionID,RegionName,StateName,SizeRank,2015-10,2015-11,2015-12,2016-01,2016-02,...,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10
0,12,25290,Jacksonville,Florida,13,98900.0,98700.0,98900.0,100100.0,102100.0,...,141900,146300,147400,153300,156700,164300,171000,173000,172500,170600.0
1,20,13121,Orlando,Florida,21,169900.0,172800.0,174700.0,176700.0,176400.0,...,239900,241000,238700,239100,240100,244200,244900,241900,241500,243600.0
2,38,12700,Miami,Florida,39,286500.0,291800.0,292100.0,289200.0,291800.0,...,308300,307300,309500,316500,318700,321700,323600,320400,326100,333500.0
3,53,41176,Tampa,Florida,54,158400.0,161400.0,157800.0,167800.0,168500.0,...,229800,227600,219900,220700,222100,229000,228400,234000,232800,240500.0
4,71,19533,Naples,Florida,72,275100.0,281300.0,287600.0,297600.0,290800.0,...,310700,309000,310500,313000,319500,319700,323100,320600,322800,321800.0


## Data Scrubbing

In [3]:
#Renaming columns 4 months before and after each hurricane
#These values will be used to do logistic regression
#Matthew (10/06-08/2016): Before 2016-06 After 2017-02
value['BMatthew'] = value['2016-06']
value['AMatthew'] = value['2017-02']

#Irma (09/10-11/2017): Before 2017-05 After 2018-01
value['BIrma'] = value['2017-05']
value['AIrma'] = value['2018-01']

#Michael (10/09-11/2018): Before 2018-06 After 2019-02 
value['BMichael'] = value['2018-06']
value['AMichael'] = value['2019-02']

#dropping unnecessary columns 
#droppping the rest of the value columns
value = value.loc[:, ~value.columns.str.startswith('2')]

#dropping unnamed: 0 and regionID because they are unique identifiers
#dropping StateName because Florida is the only state being worked with
value.drop(['Unnamed: 0','RegionID', 'StateName'], axis=1, inplace=True)

#Checking it out
value.head()

,RegionName,SizeRank,BMatthew,AMatthew,BIrma,AIrma,BMichael,AMichael
0,Jacksonville,13,111700.0,128500.0,129200.0,133900.0,140700,146300
1,Orlando,21,179800.0,191400.0,196000.0,213000.0,234800,241000
2,Miami,39,286300.0,301100.0,304100.0,312900.0,315500,307300
3,Tampa,54,187300.0,187500.0,185300.0,205100.0,211600,227600
4,Naples,72,283400.0,292600.0,299600.0,295500.0,311500,309000


In [4]:
#let's create a dataframe for each hurricane and then concat them together

#creating dataframe for hurricane Matthew (1)
df_Ma = value[['RegionName', 'SizeRank', 'BMatthew', 'AMatthew']]
#HurricaneName will be '1' 
df_Ma['HurricaneName'] = '1'
df_Ma = df_Ma.rename(columns={"BMatthew": "Before", "AMatthew": "After"})

#creating dataframe for hurricane Irma (2)
df_I = value[['RegionName', 'SizeRank', 'BIrma', 'AIrma']]
#HurricaneName will be '2' 
df_I['HurricaneName'] = '2'
df_I = df_I.rename(columns={"BIrma": "Before", "AIrma": "After"})

#creating dataframe for hurricane Michael (3)
df_Mi = value[['RegionName', 'SizeRank', 'BMichael', 'AMichael']]
#HurricaneName will be '3' 
df_Mi['HurricaneName'] = '3'
df_Mi = df_Mi.rename(columns={"BMichael": "Before", "AMichael": "After"})

#concating the three dataframes into one 
df = pd.concat([df_Ma, df_I, df_Mi], ignore_index=True)

#renaming 'RegionName' to 'City'
df = df.rename(columns={"RegionName":"City"})

#taking a look
df.head()

,City,SizeRank,Before,After,HurricaneName
0,Jacksonville,13,111700.0,128500.0,1
1,Orlando,21,179800.0,191400.0,1
2,Miami,39,286300.0,301100.0,1
3,Tampa,54,187300.0,187500.0,1
4,Naples,72,283400.0,292600.0,1


## Data Exploration 

Let's get an idea of what home value increase looks like from 4 months before and after the hurricanes. This will inform the cutoff value we use when modeling. Our Q3 value is just about 10% for all three hurricanes, so this will be the value used for cutoff.  

In [5]:
#percent = (after - before/before)*100

#finding percent change for Hurricane Matthew
df['Percent'] = (df['After'] - df['Before'])/df['Before']*100

#Q3 ~10% 
df.describe()

,SizeRank,Before,After,Percent
count,831.000000,791.000000,813.000000,791.000000
mean,1711.162455,229482.806574,241690.897909,5.961250
std,1149.142889,104078.880056,109101.900399,7.948774
min,13.000000,34100.000000,35600.000000,-21.457404
25%,691.000000,162200.000000,171200.000000,1.468005
50%,1587.000000,200800.000000,215300.000000,5.689001
75%,2715.000000,268900.000000,280600.000000,9.992094
max,3723.000000,735400.000000,836600.000000,42.339008


## Data Engineering

Using the 10% cutoff value a column will be engineered stating if 'True' there was a 10% of more increase in value, or 'False' there was less than a 10% increase in value. 

In [6]:
#1 = true, there was a 10% more increase in value 
#0 = false, there was not a 10% or more increase in value
df['Increase'] = np.where(df['Percent'] >= 10, 1, 0)
df.head()

,City,SizeRank,Before,After,HurricaneName,Percent,Increase
0,Jacksonville,13,111700.0,128500.0,1,15.040286,1
1,Orlando,21,179800.0,191400.0,1,6.451613,0
2,Miami,39,286300.0,301100.0,1,5.169403,0
3,Tampa,54,187300.0,187500.0,1,0.106781,0
4,Naples,72,283400.0,292600.0,1,3.246295,0


In [7]:
#Dropping any missing values
df = df.dropna()
df.isnull().sum()

City             0
SizeRank         0
Before           0
After            0
HurricaneName    0
Percent          0
Increase         0
dtype: int64

In [52]:
df.to_csv(r'data\housing.csv', index=False)

# Joining Housing Values with Hurricanes 

In [147]:
hurricane = pd.read_csv(r'data\hurricane.csv')
housing =  pd.read_csv(r'data\housing.csv')

In [148]:
hurricane.set_index(['HurricaneName', 'City'], inplace = True)

In [149]:
hurricane.index

MultiIndex([(1,    'Apalachicola'),
            (1,    'Apalachicola'),
            (1,    'Apalachicola'),
            (1,         'Bonifay'),
            (1,         'Bonifay'),
            (1,         'Bonifay'),
            (1,     'Brooksville'),
            (1,     'Brooksville'),
            (1,     'Brooksville'),
            (1,       'Crestview'),
            ...
            (3, 'West Palm Beach'),
            (3, 'West Palm Beach'),
            (3, 'West Palm Beach'),
            (3,          'Milton'),
            (3,          'Milton'),
            (3,          'Milton'),
            (3,          'Milton'),
            (3,    'Winter Haven'),
            (3,    'Winter Haven'),
            (3,    'Winter Haven')],
           names=['HurricaneName', 'City'], length=349)

In [150]:
housing.set_index(['HurricaneName', 'City'], inplace = True)

In [151]:
housing.index

MultiIndex([(1,         'Jacksonville'),
            (1,              'Orlando'),
            (1,                'Miami'),
            (1,                'Tampa'),
            (1,               'Naples'),
            (1,          'Tallahassee'),
            (1,              'Hialeah'),
            (1,     'Saint Petersburg'),
            (1,            'Pensacola'),
            (1,             'Sarasota'),
            ...
            (3,        'Madeira Beach'),
            (3,        'Miramar Beach'),
            (3,             'Belleair'),
            (3,   'Indian Rocks Beach'),
            (3,         'Longboat Key'),
            (3,         'Tierra Verde'),
            (3,              'Placida'),
            (3, 'Daytona Beach Shores'),
            (3,         'Holmes Beach'),
            (3,        'Indian Shores')],
           names=['HurricaneName', 'City'], length=791)

In [153]:
hurricane.head()

DATE   AWND  WSF2  WSF5
HurricaneName City                                      
1             Apalachicola  10/8/2016   8.05  15.0  21.9
              Apalachicola  10/7/2016  10.74  21.9  27.1
              Apalachicola  10/6/2016  11.63  19.9  25.1
              Bonifay       10/8/2016   6.04  15.0  21.0
              Bonifay       10/7/2016   8.28  18.1  25.1

In [154]:
housing.head()

SizeRank    Before     After    Percent  Increase
HurricaneName City                                                           
1             Jacksonville        13  111700.0  128500.0  15.040286         1
              Orlando             21  179800.0  191400.0   6.451613         0
              Miami               39  286300.0  301100.0   5.169403         0
              Tampa               54  187300.0  187500.0   0.106781         0
              Naples              72  283400.0  292600.0   3.246295         0

In [155]:
df = hurricane.join(housing, how='inner')
df

DATE   AWND  WSF2  WSF5  SizeRank  \
HurricaneName City                                                       
1             Brooksville       10/8/2016   8.05  19.9  25.1      1639   
              Brooksville       10/7/2016  19.69  30.0  38.9      1639   
              Brooksville       10/6/2016  11.63  21.9  29.1      1639   
              Clearwater        10/8/2016   8.28  17.0  21.9       223   
              Clearwater        10/7/2016  20.58  31.1  38.9       223   
...                                   ...    ...   ...   ...       ...   
3             West Palm Beach  10/10/2018  12.75  25.1  33.1       313   
              West Palm Beach   10/9/2018  13.65  23.0  28.0       313   
              Winter Haven     10/11/2018   7.38  16.1  25.1       801   
              Winter Haven     10/10/2018  13.20  25.1  33.1       801   
              Winter Haven      10/9/2018  11.63  18.1  23.0       801   

                                 Before     After   Percent  Increase  
HurricaneName City                                                     
1             Brooksville      122900.0  134300.0  9.275834         0  
              Brooksville      122900.0  134300.0  9.275834         0  
              Brooksville      122900.0  134300.0  9.275834         0  
              Clearwater       164500.0  168700.0  2.553191         0  
              Clearwater       164500.0  168700.0  2.553191         0  
...                                 ...       ...       ...       ...  
3             West Palm Beach  208100.0  206900.0 -0.576646         0  
              West Palm Beach  208100.0  206900.0 -0.576646         0  
              Winter Haven     148400.0  158600.0  6.873315         0  
              Winter Haven     148400.0  158600.0  6.873315         0  
              Winter Haven     148400.0  158600.0  6.873315         0  

[320 rows x 9 columns]

In [156]:
df.reset_index(inplace = True)
df

,HurricaneName,City,DATE,AWND,WSF2,WSF5,SizeRank,Before,After,Percent,Increase
0,1,Brooksville,10/8/2016,8.05,19.9,25.1,1639,122900.0,134300.0,9.275834,0
1,1,Brooksville,10/7/2016,19.69,30.0,38.9,1639,122900.0,134300.0,9.275834,0
2,1,Brooksville,10/6/2016,11.63,21.9,29.1,1639,122900.0,134300.0,9.275834,0
3,1,Clearwater,10/8/2016,8.28,17.0,21.9,223,164500.0,168700.0,2.553191,0
4,1,Clearwater,10/7/2016,20.58,31.1,38.9,223,164500.0,168700.0,2.553191,0
...,...,...,...,...,...,...,...,...,...,...,...
315,3,West Palm Beach,10/10/2018,12.75,25.1,33.1,313,208100.0,206900.0,-0.576646,0
316,3,West Palm Beach,10/9/2018,13.65,23.0,28.0,313,208100.0,206900.0,-0.576646,0
317,3,Winter Haven,10/11/2018,7.38,16.1,25.1,801,148400.0,158600.0,6.873315,0
318,3,Winter Haven,10/10/2018,13.20,25.1,33.1,801,148400.0,158600.0,6.873315,0


In [157]:
df.isnull().sum()

HurricaneName    0
City             0
DATE             0
AWND             0
WSF2             0
WSF5             0
SizeRank         0
Before           0
After            0
Percent          0
Increase         0
dtype: int64

## Feature Engineering 
-grouping by max 

In [167]:
df.head()

,HurricaneName,City,DATE,AWND,WSF2,WSF5,SizeRank,Before,After,Percent,Increase
0,1,Brooksville,10/8/2016,8.05,19.9,25.1,1639,122900.0,134300.0,9.275834,0
1,1,Brooksville,10/7/2016,19.69,30.0,38.9,1639,122900.0,134300.0,9.275834,0
2,1,Brooksville,10/6/2016,11.63,21.9,29.1,1639,122900.0,134300.0,9.275834,0
3,1,Clearwater,10/8/2016,8.28,17.0,21.9,223,164500.0,168700.0,2.553191,0
4,1,Clearwater,10/7/2016,20.58,31.1,38.9,223,164500.0,168700.0,2.553191,0


In [169]:
df.drop('DATE', axis = 1, inplace=True)
df

,HurricaneName,City,AWND,WSF2,WSF5,SizeRank,Before,After,Percent,Increase
0,1,Brooksville,8.05,19.9,25.1,1639,122900.0,134300.0,9.275834,0
1,1,Brooksville,19.69,30.0,38.9,1639,122900.0,134300.0,9.275834,0
2,1,Brooksville,11.63,21.9,29.1,1639,122900.0,134300.0,9.275834,0
3,1,Clearwater,8.28,17.0,21.9,223,164500.0,168700.0,2.553191,0
4,1,Clearwater,20.58,31.1,38.9,223,164500.0,168700.0,2.553191,0
...,...,...,...,...,...,...,...,...,...,...
315,3,West Palm Beach,12.75,25.1,33.1,313,208100.0,206900.0,-0.576646,0
316,3,West Palm Beach,13.65,23.0,28.0,313,208100.0,206900.0,-0.576646,0
317,3,Winter Haven,7.38,16.1,25.1,801,148400.0,158600.0,6.873315,0
318,3,Winter Haven,13.20,25.1,33.1,801,148400.0,158600.0,6.873315,0


In [177]:
df = df.groupby(['HurricaneName', 'City']).max()
df

AWND  WSF2  WSF5  SizeRank    Before  \
HurricaneName City                                                     
1             Brooksville      19.69  30.0  38.9      1639  122900.0   
              Clearwater       20.58  31.1  38.9       223  164500.0   
              Crestview         8.95  19.9  32.0       994  134700.0   
              Daytona Beach    27.51  52.1  70.9       658  109500.0   
              Destin           12.75  19.9  25.1      2830  308900.0   
...                              ...   ...   ...       ...       ...   
3             Tampa            14.99  25.1  35.1        54  211600.0   
              Tavares          13.87  35.1  49.0      2390  190700.0   
              Vero Beach       14.54  21.9  28.0       328  195400.0   
              West Palm Beach  13.65  25.1  33.1       313  208100.0   
              Winter Haven     13.20  25.1  33.1       801  148400.0   

                                  After    Percent  Increase  
HurricaneName City                                            
1             Brooksville      134300.0   9.275834         0  
              Clearwater       168700.0   2.553191         0  
              Crestview        149300.0  10.838901         1  
              Daytona Beach    117200.0   7.031963         0  
              Destin           307100.0  -0.582713         0  
...                                 ...        ...       ...  
3             Tampa            227600.0   7.561437         0  
              Tavares          196800.0   3.198741         0  
              Vero Beach       205600.0   5.220061         0  
              West Palm Beach  206900.0  -0.576646         0  
              Winter Haven     158600.0   6.873315         0  

[95 rows x 8 columns]

# Logistic Regression

In [178]:
# For our modeling steps
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss

# For demonstrative pruposes
from scipy.special import logit, expit

NOAA Data   
Average wind speed (AWND)   
Peak gust time (PGTM)     
Direction of fastest 2-minute wind (WDF2) (in degrees)   
Direction of fastest 5-second wind (WDF5) (in degrees)   
Fastest 2-minute wind speed (WSF2) (in meters per second)    
Fastest 5-second wind speed (WSF5) (in meter per second)      
24-hour wind movement (km or miles as per user preference, miles on Daily Form pdf file) (WDMV)    
Weather types (WT**)    

In [211]:
#check crosstabs 
df[df.Increase == 0].describe()

,AWND,WSF2,WSF5,SizeRank,Before,After,Percent,Increase
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.0
mean,19.563000,35.890000,48.965714,543.257143,207454.285714,213262.857143,2.818421,0.0
std,6.912737,13.285376,17.896910,731.204797,78722.879730,81678.172224,4.541435,0.0
min,10.510000,18.100000,23.000000,13.000000,109500.000000,117200.000000,-12.659123,0.0
25%,14.375000,25.300000,36.000000,89.250000,152125.000000,157850.000000,0.140577,0.0
50%,17.670000,31.100000,42.100000,291.000000,188350.000000,192950.000000,3.716528,0.0
75%,21.920000,45.300000,66.325000,655.750000,245275.000000,253525.000000,5.822047,0.0
max,40.710000,70.900000,87.900000,2830.000000,614400.000000,643100.000000,9.716387,0.0


In [212]:
df[df.Increase == 1].describe()

,AWND,WSF2,WSF5,SizeRank,Before,After,Percent,Increase
count,25.000000,25.000000,25.000000,25.00000,25.000000,25.000000,25.000000,25.0
mean,21.555200,39.052000,54.000000,509.52000,172364.000000,196208.000000,13.805276,1.0
std,9.163215,15.391126,20.807611,606.77303,77081.882221,90191.430302,3.798559,0.0
min,8.950000,19.900000,31.100000,13.00000,111700.000000,128500.000000,10.038860,1.0
25%,14.320000,25.900000,33.100000,127.00000,138900.000000,160600.000000,11.916376,1.0
50%,19.240000,33.100000,47.000000,340.00000,154400.000000,172800.000000,12.557870,1.0
75%,25.500000,52.100000,69.100000,577.00000,172900.000000,197900.000000,14.459225,1.0
max,40.260000,74.000000,91.000000,2390.00000,510100.000000,597300.000000,25.937261,1.0


In [179]:
logreg = LogisticRegression(random_state=56)

In [205]:
#y is prediction variable 
#X is features 
y = df['Increase']
X = df.drop(['Increase', 'After', 'Percent'], axis=1)

In [206]:
X.head()

AWND  WSF2  WSF5  SizeRank    Before
HurricaneName City                                                
1             Brooksville    19.69  30.0  38.9      1639  122900.0
              Clearwater     20.58  31.1  38.9       223  164500.0
              Crestview       8.95  19.9  32.0       994  134700.0
              Daytona Beach  27.51  52.1  70.9       658  109500.0
              Destin         12.75  19.9  25.1      2830  308900.0

In [207]:
logreg.fit(X, y)

LogisticRegression(random_state=56)

In [209]:
print(len(logreg.predict(X)))

95
